In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, Row
import pydeequ
import pyspark
from pyspark.sql.types import *
from pydeequ import Check,CheckLevel
from pydeequ.checks import *
from pydeequ.verification import *
from pydeequ.analyzers import *
from pyspark.sql.functions import lit,current_timestamp
from configparser import ConfigParser
import sys

In [2]:
Parser = ConfigParser()
configFile = r"C:\Users\Shivam\Desktop\training\godaddy\godaddy\config\config_analysis.json"
Parser.read(configFile)
Env = "dev"
#SourceFileName = "merchants.csv"

##Set Job Parameters
Raw_S3_Path = Parser.get(Env, 'raw_s3_path')
#Clean_S3_Path = Parser.get(Env, 'clean_s3_path')
Format = Parser.get(Env, 'file_format')
print("S3_Path = " + Raw_S3_Path)
print("Format = " + Format)
#print("SourceFile = " + S3_Path )

S3_Path = C:\\Users\\Shivam\\Desktop\\training\\godaddy\\godaddy\\data\\merchants.csv
Format = csv


In [3]:
analyzers = Parser.get(Env,'checks').split(',')
analyzers

["Completeness('id')",
 " Distinctness('id')",
 " Minimum('id')",
 " Maximum('id')",
 " Completeness('telephone')"]

In [4]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "C:\Program Files (x86)\PostgreSQL\pgJDBC\postgresql-42.2.18.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

In [5]:
df = spark.read.format(Format).load(Raw_S3_Path,header=True,inferSchema=True).select('id','state','updated_at','telephone')

In [6]:
df.show()

+------+-----------+--------------------+--------------+
|    id|      state|          updated_at|     telephone|
+------+-----------+--------------------+--------------+
| 12498|trial_ended|2018-12-19 23:43:...|   45434323212|
| 14319|trial_ended|2018-12-19 23:43:...|  413-478-8291|
|  8338|trial_ended|2018-12-19 23:44:...|   09701189483|
|130989|trial_ended|2020-08-03 17:01:...|00447921706558|
| 59774|  suspended|2019-07-07 13:07:...|  000-000-0000|
|  8284|trial_ended|2018-12-19 23:42:...|    2065126641|
|158778|trial_ended|2020-11-03 01:21:...|    9783376017|
|151006|  suspended|2020-10-12 17:12:...|  000-000-0000|
|141731|  suspended|2020-11-30 12:26:...|  000-000-0000|
| 50768|  suspended|2019-05-15 00:20:...|    7705476904|
| 36169|trial_ended|2018-12-19 23:45:...|    4243039800|
| 78396|  suspended|2020-03-01 02:13:...|    8018002701|
| 45140|  pre_trial|2019-02-11 20:39:...|    9173281930|
| 28378|trial_ended|2018-12-19 23:42:...|    5166038093|
|123728|provisioned|2020-12-02 

In [7]:
analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(eval("Size()"))

res = lambda fun : analysisResult.addAnalyzer(eval(fun))

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, list(map(res,analyzers))[0].run())
analysisResult_df.show()

+-------+---------+------------+--------+
| entity| instance|        name|   value|
+-------+---------+------------+--------+
|Dataset|        *|        Size|  1000.0|
| Column|       id|Distinctness|     1.0|
| Column|       id|Completeness|     1.0|
| Column|       id|     Minimum|   469.0|
| Column|       id|     Maximum|174577.0|
| Column|telephone|Completeness|   0.999|
+-------+---------+------------+--------+



In [ ]:
#analysisResult = AnalysisRunner(spark) \
 #                   .onData(df) \
  #                  .addAnalyzer(eval("Size()"))

#for fun in analyzers:
 #   analysisResult = analysisResult.addAnalyzer(eval(fun))
#analysisResult = analysisResult.run()
  #.addAnalyzer(CountDistinct("state")) \
                  
                   

#analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
#analysisResult_df.show()

In [8]:
analysisResult_df = analysisResult_df.withColumn('ingestion_tsp',lit(current_timestamp()))
analysisResult_df = analysisResult_df.withColumn('source_table',lit(Parser.get(Env,'data_source')))

In [9]:
analysisResult_df.show()

+-------+---------+------------+--------+--------------------+------------+
| entity| instance|        name|   value|       ingestion_tsp|source_table|
+-------+---------+------------+--------+--------------------+------------+
|Dataset|        *|        Size|  1000.0|2020-12-21 16:55:...|   merchants|
| Column|       id|Distinctness|     1.0|2020-12-21 16:55:...|   merchants|
| Column|       id|Completeness|     1.0|2020-12-21 16:55:...|   merchants|
| Column|       id|     Minimum|   469.0|2020-12-21 16:55:...|   merchants|
| Column|       id|     Maximum|174577.0|2020-12-21 16:55:...|   merchants|
| Column|telephone|Completeness|   0.999|2020-12-21 16:55:...|   merchants|
+-------+---------+------------+--------+--------------------+------------+



In [37]:
#df.write.jdbc(url="jdbc:postgresql://localhost:5433/postgres",table='public.users_table',mode='overwrite',properties={"user": "postgres", "password": "root", "driver": 'org.postgresql.Driver'})

In [10]:
analysisResult_df.write.csv('..//analyzer_files//merchant_analyzer',header=True,mode='overwrite')